### ODS层
> 操作型数据

> Operational Data Store

- 对于实时数仓来说，ods层一般是存储在kafka中的原始数据。所谓原始数据，主要有两个来源:

  - 基于canal采集到kafka中的mysql表的binlog日志
  - 基于flume/fluentd采集到kafka中的客户端访问日志
  - PC网页访问日志
  - H5端访问日志
  - 小程序访问日志
  - APP访问日志
  - 后端网关服务日志
那么，对于ODS层的数据，我们需要做哪些处理呢？通常来说，什么都不做，ods层一般会保留数据原样，方便定位与回溯数据。

---
### DWD
> 数据仓库明细层数据

> Data Warehouse Detail

- 对ODS层数据进行清洗转化，以业务过程作为建模驱动，基于每个具体的业务过程特点，构建最细粒度的明细事实表。可以结合企业的数据使用特点，基于维度建模思想，将明细事实表的某些重要属性字段做适当冗余，也即宽表化处理，构建明细宽表。主要是对ODS层的数据做一定的清洗和主题汇总。

- 以用户访问日志表为例，在dwd层一般会做的事情:

    - 保留和ODS层数据一样的数据维度
    - 包含PC、H5、小程序、APP各个来源的数据
    - 对部分枚举类型的值进行翻译
    - 剔除异常数据，保证数据质量
一般在该层，还会做表之间的join、以及维表join冗余维度信息等。

---

### DIM层设计
- 首先应该将kafka中的维度表的信息同步到数据库中存储，维度表的设计应该考虑到维度的更新、查询数据的并发以及与业务库解耦等因素，一般会用HBase或Redis来作为实时数仓DIM层的存储。

    - 对于维度变化缓慢且维度数据比较大的维度表，可以考虑用HBase存储，查询HBase时可以结合本地缓存 + 异步IO来实现。
    - 对于维度变化较快且维表数据量相对较小的维度表，一般可以用Redis存储。

### DWS
> 数据仓库汇总层数据

> Data Warehouse Summary

- 基于指标需求，构建初步汇总事实表，一般是宽表。
- 基于上层的应用和产品的指标需求，构建公共粒度的汇总指标表。
- 以宽表化手段物理化模型，构建命名规范、口径一致的统计指标，为上层提供公共指标。

---
### ADS
> 面向应用的数据服务层

> Application Data Service
```text
整合汇总成分析某一个主题域的服务数据，面向应用逻辑的数据加工。
该层主要存放数据产品个性化的统计指标数据，这一层的数据直接对接数据的消费者，是产品、运营等角色可以直接感知理解的一层，大多数这一层的表都可以直接在BI上通过图表的形式直接透出。
```